<a href="https://colab.research.google.com/github/abdullahnizami77/inshorts-scraping/blob/main/inshorts_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modular un-updated code

In [ ]:
import urllib.request
import urllib
import urllib.parse
from bs4 import BeautifulSoup
from urllib.parse import quote
import os
import time
import sys
import random
import string
import re

def myquote(quote_page):
    url = urllib.parse.urlsplit(quote_page)
    url = list(url)
    url[2] = urllib.parse.quote(url[2])
    url = urllib.parse.urlunsplit(url)
    return url

try:
    if "hindi" in str(sys.argv[1]).lower():
        file_name = "/content/links.txt"
        destination_path = "/content/dataset/hindi/"
        full_ds_path = "/content/inshorts-dataset-hi/"
    else:
        file_name = "/content/links.txt"
        destination_path = "/content/dataset/english/"
        full_ds_path = "/content/inshorts-dataset-en/"

except:
    print("\tPlease provide 'Hindi' or 'English' after the command. Exit and run again.")
    input()
    exit()

os.makedirs(destination_path, exist_ok=True)
os.makedirs(full_ds_path, exist_ok=True)

linkfile = open(file_name, "r")
links = linkfile.readlines()

i = 0
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers = {'User-Agent': user_agent}
k = 1

display_top_n = 5

for link_number, quote_page in enumerate(links, start=1):
    quote_page = quote_page.strip()

    if quote_page.count('http') > 1:
        continue

    print(f"Processing link {link_number}/{len(links)}: {quote_page}")

    try:
        page = urllib.request.urlopen(myquote(quote_page))
    except Exception as e:
        print(e)
        input('wait1')
        continue

    if "FILE_TERMINATES_HERE" in quote_page:
        break

    rstring = ''.join(random.choices(string.ascii_uppercase, k=4))
    dtstring = os.path.join(destination_path, re.search(r'\d+$', quote_page)[0] + rstring + ".txt")
    dtstring2 = os.path.join(full_ds_path, re.search(r'\d+$', quote_page)[0] + rstring + ".txt")

    file = open(dtstring, 'w')
    article_len = 0

    try:
        soup = BeautifulSoup(page, 'html.parser')
        selected_div = None
        for div in soup.findAll('div'):
            if div.text and 'read more at' in div.find(text=True):
                selected_div = div
                break
        urlink = selected_div.find("a", target="_blank")['href']
        request = urllib.request.Request(urlink, None, headers)
        response = urllib.request.urlopen(request)
        soup2 = BeautifulSoup(response, 'html.parser')
        file.write("#originalArticleHeadline" + "\n")

        if "hindustantimes.com" in urlink:
            hlist = soup2.find('h1')
            hlist2 = soup2.find('h2')
            file.write(hlist.text + "\n")
            file.write(hlist2.text + "\n")

            file.write("#originalArticleBody" + "\n")
            table = soup2.findAll('div', attrs={"class": "story-content"})
            for x in table:
                for p in x.findAll('p'):
                    file.write(p.text + "\n")

        elif "inextlive.com" in urlink:
          hlist = soup2.find('h1')
          file.write("#originalArticleHeadline" + "\n")
          file.write(hlist.text + "\n")
          file.write("#originalArticleBody" + "\n")
          article = soup2.findAll('p')
          for a in article:
            if "Copyright © 2023. All Rights Reserved" not in a.text:
              file.write(a.text + "\n")


        elif "amarujala.com" in urlink:
            hlist = soup2.find('h1')
            #hlist2 = soup2.find('h2')
            file.write(hlist.text + "\n")
            #file.write(hlist2.text + "\n")

            file.write("#originalArticleBody" + "\n")
            table = soup2.findAll('div', attrs={"class": "article-desc ul_styling"})
            for x in table:
                for p in x.findAll('p'):
                    file.write(p.text + "\n")

        # I Will Add more website-specific parsing logic here as needed

        else:
            hlist = soup2.find('h1')
            file.write(hlist.text + "\n")
            file.write("#originalArticleBody" + "\n")
            article = soup2.findAll('p')
            for a in article:
                file.write(a.text + "\n")
    except Exception as e:
        print('Abdullah Error', e)
        #input('wait')
        file.close()
        os.remove(dtstring)
        print('  --> Waiting of 1 sec. Press Ctrl+C to exit', dtstring, ' ' * 90, end='\r')
        time.sleep(1)
        continue

    file.write("\n")
    file.write("-" * 100 + "\n")
    file.write("#summaryHeadline\n" + soup.find("span", itemprop="headline").text.strip() + "\n")
    file.write("#summaryBody\n" + soup.find("div", itemprop="articleBody").text.strip() + "\n")
    file.write("#datePublished " + soup.find("span", itemprop="datePublished").text.strip() + " ")
    file.write(soup.find("span", clas="date").text.strip() + "\n")
    #file.write(soup.find("span", attrs={'class': 'short'}).text.strip() + " by " + soup.find("span", attrs={
        #'class': 'author'}).text.strip() + " from News inShorts\n")
    file.write("#reference_link: " + quote_page + "\n")
    file.write("#original_link: " + urlink)
    file.close()

    print('dtstring', dtstring)
    #input('wait')

    print(str(link_number) + " " + quote_page[29:-14] + "\n    Article and Summary pulled!" + " " + datetime.datetime.now().strftime(
        "%H:%M:%S") + ' ' * 100 + '\n    file ' + dtstring2 + ' ' + " " * 100 + '\n' + '-' * 50 + ' ' * 100)
    if (k % display_top_n == 0):
        print("\033[A" * (display_top_n * 4), end='')

    k += 1

print("\n" * 7 + str(k - 1) + " Articles and their summaries pulled!")


Processing link 1/20: https://inshorts.com/hi/news/डेरा-सच्चा-सौदा-प्रमुख-गुरमीत-राम-रहीम-सिंह-अनुयायी-की-हत्या-के-लिए-दोषी-करार-1633677650409


<ipython-input-4-e28e4098310b>:77: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  if div.text and 'read more at' in div.find(text=True):


dtstring /content/dataset/english/1633677650409KHMW.txt
1 डेरा-सच्चा-सौदा-प्रमुख-गुरमीत-राम-रहीम-सिंह-अनुयायी-की-हत्या-के-लिए-दोषी-करार
    Article and Summary pulled! 19:07:58                                                                                                    
    file /content/inshorts-dataset-en/1633677650409KHMW.txt                                                                                                     
--------------------------------------------------                                                                                                    
Processing link 2/20: https://inshorts.com/hi/news/सहारनपुर-में-बसपा-के-फज़लुर्रहमान-सबसे-आगे-मसूद-को-भी-मिल-चुके-सवा-लाख-वोट-1558601614502
dtstring /content/dataset/english/1558601614502PRSH.txt
2 सहारनपुर-में-बसपा-के-फज़लुर्रहमान-सबसे-आगे-मसूद-को-भी-मिल-चुके-सवा-लाख-वोट
    Article and Summary pulled! 19:07:59                                                                                                  

# Exploratory Code

In [ ]:
import urllib.request
import urllib
import urllib.parse
from bs4 import BeautifulSoup
from urllib.parse import quote
import datetime
import os
import time
import sys
import random, string, re

In [ ]:
def encode_hindi_url(quote_page):
    url = urllib.parse.urlsplit(quote_page)
    url = list(url)
    url[2] = urllib.parse.quote(url[2])
    url = urllib.parse.urlunsplit(url)
    return url

input_file = "links.txt"

try:
    file = open(input_file, "r", encoding="utf-8")
    links = file.readlines()
    file.close()
except FileNotFoundError:
    print(f"File '{input_file}' not found.")
    exit()


output_directory = "scraped_headings/"


if not os.path.exists(output_directory):
    os.makedirs(output_directory)


for i, link in enumerate(links[:5]):  # Only top 5 links
    link = link.strip()
    print(f"Scraping link {i + 1}: {link}")


    encoded_link = encode_hindi_url(link)

    try:
        page = urllib.request.urlopen(encoded_link)
    except Exception as e:
        print(f"Error fetching {encoded_link}: {e}")
        continue

    soup = BeautifulSoup(page, 'html.parser')


    heading = soup.find("span", itemprop="headline")

    if heading:
        heading_text = heading.text.strip()

        filename = os.path.join(output_directory, f"heading_{i + 1}.txt")

        with open(filename, "w", encoding="utf-8") as file:
            file.write(heading_text)

        print(f"Heading {i + 1} saved.")

    time.sleep(random.uniform(1, 3))

print(f"Scraping complete for the top 5 links. Headings saved in '{output_directory}'")

Scraping link 1: https://inshorts.com/hi/news/डेरा-सच्चा-सौदा-प्रमुख-गुरमीत-राम-रहीम-सिंह-अनुयायी-की-हत्या-के-लिए-दोषी-करार-1633677650409
Heading 1 saved.
Scraping link 2: https://inshorts.com/hi/news/सहारनपुर-में-बसपा-के-फज़लुर्रहमान-सबसे-आगे-मसूद-को-भी-मिल-चुके-सवा-लाख-वोट-1558601614502
Heading 2 saved.
Scraping link 3: https://inshorts.com/hi/news/गुरुग्राम-में-फ्लैट-के-नाम-पर-दिल्ली-के-व्यापारी-से-₹9125-की-ठगी-1556099842849
Heading 3 saved.
Scraping link 4: https://inshorts.com/hi/news/अपने-आत्मीय-मित्र-को-याद-कर-रहा-हूं-पासवान-की-बरसी-पर-उनके-परिवार-को-संदेश-में-पीएम-1631439024472
Heading 4 saved.
Scraping link 5: https://inshorts.com/hi/news/मुज़फ्फरनगर-में-प्रेम-प्रसंग-के-चलते-हमलावरों-ने-की-युवक-की-गोली-मारकर-हत्या-1559907245740
Heading 5 saved.
Scraping complete for the top 5 links. Headings saved in 'scraped_headings/'


# Amar Ujala (Working + Non Modular)

In [ ]:
from bs4 import BeautifulSoup
import requests
import re



In [ ]:
url = 'https://www.amarujala.com/delhi-ncr/gurgaon/21556042696-gurgaon-news?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()

soup = BeautifulSoup(response.content, 'html.parser')


In [ ]:
headlines = soup.find_all('h1')

for headline in headlines:
    print("Headline:", headline.text.strip())

Headline: फलैट के नाम पर जाली कागजात बनाकर व्यापारी से 91.25 की धोखाधड़ी


In [ ]:
article_div = soup.find('div', class_='article-desc tested')
if article_div is None:
    article_div= soup.find('div', class_='article-desc ul_styling')

if article_div:
    article_text = article_div.get_text(separator="\n")
    print(article_text)

else:
    print("Article content not found on the page.")






                                                                                 
                जाली कागजात बनाकर व्यापारी से 91.25 लाख की धोखाधड़ी 
                
                
                 
                    






 
यह वीडियो/विज्ञापन हटाएं



                
                
                

                                                
                
                                                                                                
                                 
                क्रॉसर...
                
                
                

                                                
                
                                                                                                
                                 
                

                
                
                

                                                
                
                                                                           

In [ ]:
cleaned_article = ' '.join(article_text.split())
cleaned_article = '\n'.join(line.strip() for line in cleaned_article.splitlines() if line.strip())
cleaned_article = re.sub(r'\s+([.,;!?])', r'\1', cleaned_article)
print(cleaned_article)







जाली कागजात बनाकर व्यापारी से 91.25 लाख की धोखाधड़ी यह वीडियो/विज्ञापन हटाएं क्रॉसर... - निर्माणाधीन को-ऑपरेटिव हाउसिंग सोसाइटी में फ्लैट दिलाने के नाम पर की ठगी अमर उजाला ब्यूरो गुरुग्राम। सेक्टर-43 स्थित निर्माणाधीन को-ऑपरेटिव हाउसिंग सोसाइटी के नाम पर जाली कागजात बनाकर दिल्ली के व्यापारी से 91.25 लाख की धोखाधड़ी करने का मामला सामने आया है। पीड़ित व्यापारी से आरोपी ने दो माह के भीतर बैंक और नकदी के माध्यम से रुपये लिए। इसी बीच सोसाइटी के शेयर सर्टिफिकेट नंबर-8 की जांच कराने पर फर्जीवाड़े का खुलासा हुआ। पीड़ित ने सुशांतलोक थाना पुलिस में प्रवीण कुमार के खिलाफ धोखाधड़ी का मामला दर्ज कराया है। दिल्ली सराय रोहिल्ला स्थित विवेकानंद पुरी निवासी रणबीर सिंह ने पुलिस को दी शिकायत में बताया कि उन्होंने गुरुग्राम में फ्लैट खरीदने के लिए अपने साढू जयपाल सिंह के माध्यम से गुरुग्राम सेक्टर-43 रैनबो अपार्टमेंट फ्लैट नबंर-1202 निवासी प्रवीण गुप्ता से मुलाकात की। प्रवीण ने उनको बताया कि सेक्टर-43 स्थित निर्माणाधीन द बजरंग को-ऑपरेटिव ग्रुप हाउसिंग सोसाइटी लिमिटेड में उसके और उसकी बुआ पवित्रा के नाम से

# Cricket Tracker (Working + Non Modular)

In [ ]:
url = 'https://hindi.crictracker.com/fans-trolled-riyan-parag-on-social-media-for-a-catch/?fbclid=IwAR0YCLRo3_jNv-2rVr0vMZD2Q7iR8qVoZDJ6cgbr4m4D0YydqxN9tKtUGH0&amp=&utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()

soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
headlines = soup.find_all('h1')

for headline in headlines:
    print("Headline:", headline.text.strip())

Headline: कैच पकड़ने के बाद रियान पराग की घटिया हरकत देख भड़के फैन्स


In [ ]:
paragraphs = soup.find_all('p')


hindi_text = ""
for p in paragraphs:
    if any('\u0900' <= c <= '\u097F' for c in p.get_text()):
        hindi_text += p.get_text() + "\n"

print(hindi_text)

Prashantउप-संपादक
अद्यतन - मई 16, 2022 12:01 अपराह्न
राजस्थान टीम के युवा खिलाड़ी रियान पराग हमेशा खबरों में रहते हैं, राजस्थान के मैच में इनका प्रदर्शन अच्छा हो या नहीं ये सुर्खियां बटोर ही लेते हैं। रियान पराग पर राजस्थान टीम काफी भरोसा करती है, इसलिए सालों से रियान इस टीम के साथ जुड़े हुए हैं। लेकिन कल रात LSG के खिलाफ हुए मैच के दौरान कुछ ऐसा हो गया, जिसे देख क्रिकेट फैन्स काफी गुस्सा हो गए और रियान पराग को सोशल मीडिया पर काफी बुरी तरह से ट्रोल कर दिया।
रियान पराग गेंद और बल्ले दोनों से कमाल करना जानते हैं, लेकिन वो लगातार शानदार प्रदर्शन करने में फेल हो जाते हैं। वहीं दूसरी ओर फील्डिंग में इस युवा खिलाड़ी का कोई जवाब नहीं है, कल रात लखनऊ सुपर जायंट्स के खिलाफ हुए मैच में भी पराग ने इस क्षेत्र में अपना दम दिखाया। जहां उन्होंने बोल्ट के साथ मिलकर एक शानदार कैच पकड़ा, वहीं जब उन्होंने एक और कैच पकड़ा तो उन्होंने कुछ ऐसा कर दिया जो फैन्स को पसंद नहीं आया।
*रियान पराग ने पकड़ा था मार्कस स्टोइनिस का एक कैच।
*लेकिन गेंद रियान के हाथ में आने से पहले जमीन पर लग गई थी।
*तीसरे अंपायर ने फिर 

In [ ]:
hindi_pattern = re.compile(r'[\u0900-\u097F]+')
hindi_text_matches = hindi_pattern.findall(hindi_text)
hindi_text = ' '.join(hindi_text_matches)
print(hindi_text)

उप संपादक अद्यतन मई अपराह्न राजस्थान टीम के युवा खिलाड़ी रियान पराग हमेशा खबरों में रहते हैं राजस्थान के मैच में इनका प्रदर्शन अच्छा हो या नहीं ये सुर्खियां बटोर ही लेते हैं। रियान पराग पर राजस्थान टीम काफी भरोसा करती है इसलिए सालों से रियान इस टीम के साथ जुड़े हुए हैं। लेकिन कल रात के खिलाफ हुए मैच के दौरान कुछ ऐसा हो गया जिसे देख क्रिकेट फैन्स काफी गुस्सा हो गए और रियान पराग को सोशल मीडिया पर काफी बुरी तरह से ट्रोल कर दिया। रियान पराग गेंद और बल्ले दोनों से कमाल करना जानते हैं लेकिन वो लगातार शानदार प्रदर्शन करने में फेल हो जाते हैं। वहीं दूसरी ओर फील्डिंग में इस युवा खिलाड़ी का कोई जवाब नहीं है कल रात लखनऊ सुपर जायंट्स के खिलाफ हुए मैच में भी पराग ने इस क्षेत्र में अपना दम दिखाया। जहां उन्होंने बोल्ट के साथ मिलकर एक शानदार कैच पकड़ा वहीं जब उन्होंने एक और कैच पकड़ा तो उन्होंने कुछ ऐसा कर दिया जो फैन्स को पसंद नहीं आया। रियान पराग ने पकड़ा था मार्कस स्टोइनिस का एक कैच। लेकिन गेंद रियान के हाथ में आने से पहले जमीन पर लग गई थी। तीसरे अंपायर ने फिर दे दिया था मार्कस को नॉट आउट। लेकिन बा

# Timesnowhindi (working + Non Modular)

In [ ]:
url = 'https://www.timesnowhindi.com/amp/india/article/rahul-gandhi-appeals-to-party-leaders-and-workers-not-to-celebrate-his-birthday/416284?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()

soup = BeautifulSoup(response.content, 'html.parser')

headlines = soup.find_all('h1')

for headline in headlines:
    print("Headline:", headline.text.strip())


article_text = ""
paragraphs = soup.find_all("p")
for paragraph in paragraphs:
  article_text += paragraph.get_text() + "\n"
print(article_text)


Headline: Rahul Gandhi Birthday:52 के हुए राहुल गांधी, बर्थडे पर की अपील-'युवा परेशान हैं, मेरे जन्मदिन पर जश्न न मनाएं'
नयी दिल्ली: कांग्रेस नेता राहुल गांधी (Rahul Gandhi) ने पार्टी नेताओं और कार्यकर्ताओं से संडे को उनका जन्मदिन नहीं मनाने की अपील की है।राहुल ने पार्टी नेताओं और कार्यकर्ताओं से एक संदेश में कहा कि देश के युवा परेशान हैं और सड़कों पर विरोध कर रहे हैं, कांग्रेस कार्यकर्ताओं को उनके साथ खड़ा होना चाहिए, कांग्रेस के पूर्व अध्यक्ष राहुल गांधी रविवार को 52 साल (Rahul Gandhi turned 52) के हो गए हैं।
अखिल भारतीय कांग्रेस कमेटी (AICC) के महासचिव (संचार) जयराम रमेश द्वारा ट्विटर पर पोस्ट किए गए एक संदेश में राहुल गांधी ने कहा कि देश में माहौल इस समय बेहद चिंताजनक है।
राहुल ने सशस्त्र सेनाओं में भर्ती के लिए 'अग्निपथ' योजना के खिलाफ देश के कई हिस्सों में हो रहे विरोध प्रदर्शनों की ओर इशारा करते हुए कहा, 'देश के युवा परेशान हैं। हमें इस समय उनके और उनके परिवारों के साथ खड़ा होना चाहिए।'
उन्होंने कहा, 'मैं देश भर के सभी कांग्रेस कार्यकर्ताओं और अपने शुभचिंतकों से अपील करता हूं कि

#livehindustan (Not working)

In [ ]:
url = 'https://www.livehindustan.com/cricket/story-ipl-2021-david-warner-posts-selfie-from-stands-in-dubai-watches-video-srh-vs-kkr-match-4730766.amp.html?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()

soup = BeautifulSoup(response.content, 'html.parser')

Failed to retrieve the webpage.


In [ ]:
headlines = soup.find_all('h1')

for headline in headlines:
    print("Headline:", headline.text.strip())

Headline: Access Denied


# Jagran (Working + Non Modular)

In [ ]:
url = 'https://www.jagran.com/bihar/katihar-inveastigation-19191026.html?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()

soup = BeautifulSoup(response.content, 'html.parser')

headlines = soup.find_all('h1')

for headline in headlines:
    print("Headline:", headline.text.strip())

article_div = soup.find('div', class_='articlecontent')

if article_div:
    article_text = article_div.get_text(separator="\n")
    #print(article_text)
else:
    print("Article content not found on the page.")

cleaned_article = ' '.join(article_text.split())
cleaned_article = '\n'.join(line.strip() for line in cleaned_article.splitlines() if line.strip())
cleaned_article = re.sub(r'\s+([.,;!?])', r'\1', cleaned_article)
print(cleaned_article)


Headline: सूर्यास्त बाद स्टीमर परिचालन की सूचना पर औचक निरीक्षण
कटिहार। मनिहारी व साहेबगंज के बीच गंगा में सूर्यास्त के बाद स्टीमर परिचालन की सूचना पर गुरुवार की देर शाम अनुमंडल पदाधिकारी संदीप कुमार, भुमि सुधार उपसमाहर्ता रविकांत सिन्हा व सीओ संजीव कुमार ने मनिहारी लंच घाट पहुंच औचक निरीक्षण किया। कुछ देर बाद मनिहारी थाना के अनि रूपेश रंजन व अनि नवनीत कुमार भी पहुंचे। एसडीओ ने लंच घाट पर कई बातों की जानकारी भी ली। एसडीओ संदीप कुमार ने कहा कि गंगा नदी मार्ग में मनिहारी क्षेत्र में अवैध जहाजों के परिचालन पर सख्त कार्रवाई की जाएगी। किसी कीमत पर इसे बर्दाश्त नहीं किया जाएगा। उन्होंने कहा कि सूर्यास्त के बाद जहाजों व नावों का परिचालन हर हाल में वर्जित है। इसके उल्लघंन करने पर सख्त कार्रवाई की जाएगी। उन्होंने सरकार के तय निर्देश व बिहार बंगाल फेरी एक्ट का हर हाल में पालन करने का निर्देश बंदोबस्तधारी को दिया। एसडीओ ने सीओ के प्रति नाराजगी भी जताई और उन्हें आवश्यक दिशा- निर्देश भी दिया। यद्यपि ंऔचक निरीक्षण के क्रम में लंच घाट मनिहारी पर एक भी जहाज नहीं था। बताया गया कि जहाज साहिबगंज समदा घाट

# repository.inshorts.com ( Website Unavailabe)

In [ ]:
if "repository.inshorts.com" in urlink:
  print('Reference website Unavailabe')


# Jansatta (Working + Non Modular)

In [ ]:
url = 'https://www.jansatta.com/entertainment/nadav-lapid-calls-the-kashmir-files-propaganda-vivek-agnihotri-and-swara-bhaskar-react/2527781/lite/?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()
soup = BeautifulSoup(response.content, 'html.parser')
headlines = soup.find_all('h1')
for headline in headlines:
    print("Headline:", headline.text.strip())

article_div = soup.find('div', class_='post-content')

if article_div:
    article_text = article_div.get_text(separator="\n")
    #print(article_text)
else:
  print("Article content not found on the page.")

cleaned_article = ' '.join(article_text.split())
cleaned_article = '\n'.join(line.strip() for line in cleaned_article.splitlines() if line.strip())
cleaned_article = re.sub(r'\s+([.,;!?])', r'\1', cleaned_article)
print(cleaned_article)

Headline: The Kashmir Files विवाद पर बोलीं स्वरा भास्कर- दुनिया को सच्चाई पता चल गई, विवेक अग्निहोत्री ने दिया ऐसा जवाब
इंटरनेशनल फिल्म फेस्टिवल ऑफ इंडिया (IFFI) के जूरी हेड नदव लापिड (Nadav Lapid) ने फिल्म ‘द कश्मीर फाइल्स’ को घटिया और प्रोपेगेंडा बताकर एक बड़े विवाद को जन्म दे दिया है। हालांकि इजरायल के एंबेसडर नौर गिलौन (Israeli Ambassador Naor Gilon) ने भी प्रतिक्रिया देते हुए इसकी निंदा की है और अब फिल्म द कश्मीर फाइल्स के डायरेक्टर विवेक अग्निहोत्री (Vivek Agnihotri) और अभिनेत्री स्वरा भास्कर (Swara Bhaskar) ने भी इस पर प्रतिक्रिया दी है! विवेक अग्निहोत्री ने दी ये प्रतिक्रिया विवेक अग्निहोत्री (Vivek Agnihotri) ने जूरी हेड नदव लापिड (Nadav Lapid) के बयान और उस पर खड़े हुए विवाद का जिक्र नहीं किया, लेकिन उन्होंने ट्वीट किया, ”सुप्रभात। सच सबसे खतरनाक चीज है। यह लोगों से झूठ बुलवा सकता है।” विवेक अग्निहोत्री के इस ट्वीट को जूरी हेड नदव लापिड के बयान पर खड़े हुए विवाद से जोड़ कर देखा जा रहा है! स्वरा भास्कर ने किया ये ट्वीट वहीं अभिनेत्री स्वरा भास्कर (Swara Bhaskar), जो फिल्म ‘द कश्मी

# zeenews.india.com Non Working

In [ ]:
url = 'https://zeenews.india.com/hindi/india/terrorism-cartel-abolished-in-kasmir/732583/amp?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()
soup = BeautifulSoup(response.content, 'html.parser')
headlines = soup.find_all('h1')
for headline in headlines:
    print("Headline:", headline.text.strip())

Failed to retrieve the webpage.
Headline: Error 403 not available now


# India.com (working + Non Modular)

In [ ]:
url = 'https://www.india.com/hindi-news/chhattisgarh/chhattisgarh-one-woman-naxalite-killed-in-encounter-in-bastar-district-4750136/amp/?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()
soup = BeautifulSoup(response.content, 'html.parser')
headlines = soup.find_all('h1')
for headline in headlines:
    print("Headline:", headline.text.strip())
hindi_text=""
paragraphs = soup.select('.article-details p')
for p in paragraphs:
    #print(p)
    #print("----------------------------------------")
    if "ब्रेकिंग न्यूज और  लाइव न्यूज  अपडेट के लिए हमें" in p.get_text() or "More Hindi-news News" in p.get_text():
        break
    elif any('\u0900' <= c <= '\u097F' for c in p.get_text()):
        hindi_text += p.get_text() + "\n"


cleaned_hindi_text = ' '.join(hindi_text.split())
cleaned_hindi_text = '\n'.join(line.strip() for line in cleaned_hindi_text.splitlines() if line.strip())
cleaned_hindi_text = re.sub(r'\s+([.,;!?])', r'\1', cleaned_hindi_text)

print(cleaned_hindi_text)

Headline: Chhattisgarh: बस्‍तर में एनकाउंटर में महिला नक्सली ढेर, AK-47 समेत कई हथियार बरामद
रायपुर: छत्तीसगढ़ (Chhattisgarh) के नक्सल प्रभावित बस्तर जिले (Naxal-affected Bastar district) में सुरक्षा बलों के साथ मुठभेड़ (encounter) में एक महिला नक्सली मारी (woman Naxalite killed) गई. एनकाउंटर के बाद घटनास्‍थल से वर्दीधारी एक महिला नक्सली का शव(Body of a female Naxal along with arms and ammunition recovered), एक एके-47 राइफल, दो पिस्तौल, 12 बोर की बंदूक, और भारी मात्रा में अन्य सामान बरामद किया गया है. सुरक्षा बल के जवान आज शुक्रवार को सुबह करीब 8 बजे जब चांदामेटा और पयारभांट गांव के बीच जंगल में थे, तब नक्सलियों ने बस्तर डीआरजी के जवानों पर गोलीबारी शुरू कर दी. इसके बाद मठभेड़ शुरू हो गई. कुछ देर तक दोनों ओर से गोलीबारी के बाद नक्सली वहां से भाग गए. बाद में जब सुरक्षा बलों ने घटनास्थल की तलाशी ली, तब वहां वर्दीधारी एक महिला नक्सली का शव, एक एके-47 राइफल, दो पिस्तौल, 12 बोर की बंदूक, और भारी मात्रा में अन्य सामान बरामद हुआ.


# Money Control (working+ Non Modular)

In [ ]:
url = 'https://hindi.moneycontrol.com/news/india/trends/gennova-pharma-makes-vaccine-for-omicron-govt-to-review-india-s-1st-strainspecific-covid-vax-402551.html/amp?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()
soup = BeautifulSoup(response.content, 'html.parser')
headlines = soup.find_all('h1')
for headline in headlines:
    print("Headline:", headline.text.strip())
article = soup.findAll('p')
text=" "
for a in article:
  if "First Published" in a.get_text():
    break
  else:
    text+=a.get_text()
  #print(a)
  #print("-------------------")
print(text)

Headline: Omicron के लिए वैक्सीन? Gennova Pharma बना रही पहली स्ट्रेन केंद्रित कोविड वैक्सीन, सरकार करेगी रिव्यू
 Gennova Pharma : पुणे की फार्मास्युटिकल कंपनी जिनोवा बायोफार्मास्युटिकल्स लि. ने अपनी कोविड-19 मेसेंजर या एमआरएनए वैक्सीन का तीसरे चरण का ट्रायल लगभग पूरा कर लिया है। यहां तक कि कंपनी ने इसी टेक्नोलॉजी का इस्तेमाल करते हुए ओमीक्रोन स्पेसिफिक वैक्सीन (Omicron-specific vaccine) का विकास शुरू कर दिया है। टाइम्स ऑफ इंडिया की एक रिपोर्ट के मुताबिक, जिनोवा ने ओमीक्रोन वैरिएंट के लिए एमआरएनए वैक्सीन विकसित की है, जिसे सुरक्षा और प्रभाव के परीक्षण के लिए इंसानों में इस्तेमाल किया जाएगा।कैसे काम करती है यह वैक्सीनमेसेंजर आरएनए या एमआरएनए टेक्नोलॉजी में इम्यून रिस्पांस को बढ़ाने के लिए वायरस के जेनेटिक कोड (आरएनए) के एक छोटे से हिस्से को शरीर में प्रवेश कराया जाता है। इसमें प्रोटीन बनाने के लिए मानव कोशिकाओं के लिए निर्देश होते हैं, जिसमें नोवेल कोरोनावायरस एक हिस्से की नकल करते हैं और इससे इम्यून सिस्टम सक्रिय हो जाता है। वैक्सीन में कोई वास्तविक वायरस शामिल नहीं होता है।PF: जानें क

# lokmatnews. in( Working + Non Modular)


In [ ]:
url = 'https://inshorts.com/hi/news/पूर्व-अकाली-दल-नेता-ने-ट्विटर-को-भेजा-लीगल-नोटिस-कंगना-का-अकाउंट-डिलीट-करने-को-कहा-1612373514537'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()
soup = BeautifulSoup(response.content, 'html.parser')
headlines = soup.find_all('h1')
for headline in headlines:
    print("Headline:", headline.text.strip())

paragraphs = soup.select('.article-contentText p')
article_text = ""
for paragraph in paragraphs:
  article_text += paragraph.get_text() + "\n"
print(article_text)


# hindi.latestly.com (Working + Non Modular)

In [ ]:
url = 'https://www.lokmatnews.in/india/santosh-gangwar-may-be-appointed-pro-tem-speaker-for-17th-lok-sabha/?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()
soup = BeautifulSoup(response.content, 'html.parser')
headlines = soup.find_all('h1')
for headline in headlines:
    print("Headline:", headline.text.strip())
paragraphs = soup.select('article p')
article_text = ""
for paragraph in paragraphs:
  #print(paragraph)
  #print(" -----------------------------------")
  if "(यह सिंडिकेटेड न्यूज़ फीड से अनएडिटेड और ऑटो-जेनरेटेड स्टोरी है, ऐसी संभावना है कि लेटेस्टली स्टाफ द्वारा इसमें कोई बदलाव या एडिट नहीं किया गया है)" in paragraph.get_text():
    break
  else:
    article_text += paragraph.get_text() + "\n"
print(article_text)

Headline: संतोष गंगवार बन सकते हैं प्रोटेम स्पीकर, आठवीं बार चुनाव जीतकर पहुंचे हैं लोकसभा
 By रामदीप मिश्रा | Published: May 30, 2019 09:29 AM2019-05-30T09:29:25+5:302019-05-30T09:29:25+5:30
भारतीय जनता पार्टी (बीजेपी) के नेता और वरिष्ठतम लोकसभा सदस्य संतोष गंगवार को 17वीं लोकसभा के लिए प्रोटेम स्पीकर नियुक्त किया जा सकता है। हालांकि उनके साथ सुल्तानपुर से पार्टी की सांसद मेनका गांधी का भी नाम आगे चल रहा है, लेकिन कहा जा रहा है कि उन्हें मंत्रिमंडल में जगह मिल सकती है। ऐसे में संतोष गंगवार को प्रोटेम स्पीकर बनाया जा सकता है। संतोष गंगवार और मेनका गांधी दोनों ही आठ बार चुनाव जीतकर लोकसभा पहुंचे हैं।
आज शुक्रवार को होने वाले शपथ ग्रहण समारोह में यदि संतोष गंगवार और मेनका गांधी दोनों को कैबिनेट में शामिल किया जाता है, तो केरल के कांग्रेस सांसद कोडिकुन्निल सुरेश को प्रोटेम स्पीकर बनाया जा सकता है। 16वीं लोकसभा के लिए कांग्रेस के नेता कमलनाथ को प्रोटेम स्पीकर नियुक्ति किया गया था। 
प्रोटेम स्पीकर सबसे वरिष्ठ सांसद को बनाया जाता है. इसके बाद लोकसभा स्पीकर और डिप्टी स्पीकर का चुनाव होगा. ये 

# Indiatv.in (Working + Non Modular)

In [ ]:
url = 'https://www.indiatv.in/amp/india/national/kk-venugopal-has-agreed-to-continue-as-attorney-general-for-3-more-months-2022-06-29-861174?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()
soup = BeautifulSoup(response.content, 'html.parser')
headlines = soup.find_all('h1')
for headline in headlines:
    print("Headline:", headline.text.strip())

paragraphs = soup.select('p')
article_text = ""
for paragraph in paragraphs:
  #print(paragraph)
  #print(" -----------------------------------")
  if any('\u0900' <= c <= '\u097F' for c in paragraph.get_text()):
    article_text += paragraph.get_text() + "\n"
print(article_text)

Headline: K. K. Venugopal: अटॉर्नी जनरल पद पर 3 महीने और बने रहने के लिए केके वेणुगोपाल ने दी सहमति
K. K. Venugopal: वरिष्ठ अधिवक्ता के. के. वेणुगोपाल तीन और महीनों के लिए अटॉर्नी जनरल के रूप में बने रहने के लिए सहमत हैं। उनका कार्यकाल 30 जून, 2022 को समाप्त हो रहा है। केंद्र सरकार ने वरिष्ठ अधिवक्ता केके वेणुगोपाल से भारत के अटॉर्नी जनरल के रूप में बने रहने का अनुरोध किया। उनका स्वीकृत कार्यकाल 30 जून, 2022 तक है। केंद्र सरकार के अनुरोध पर वरिष्ठ अधिवक्ता केके वेणुगोपाल भारत के अटॉर्नी जनरल के रूप में बने रहने के लिए सहमत हुए। सूत्रों के अनुसार वेणुगोपाल उम्र के चलते एकबारगी इस पद से अलग होना चाहते थे। लेकिन यह भी कहा जा रहा था कि वर्तमान हालातों में सरकार के लिए वेणुगोपाल इस पद के लिए सबसे उपयुक्त व्यक्ति हैं। इसके चलते सरकार एक बार फिर उन्हें ही इस पद पर बनाए रखना चाहेगी।  
पिछली साल केंद्र सरकार ने के. के. वेणुगोपाल को दूसरी बार सेवा विस्तार दिया था। केंद्र सरकार ने उनका कार्यकाल एक साल के लिए बढ़ा दिया था। वेणुगोपाल को पीएम मोदी के नेतृत्व वाली केंद्र सरकार ने 15वें अटॉर्नी जनरल क

# hindi.news24online.com ( Working + Non Modular)

In [ ]:
url = 'https://hindi.news24online.com/state/gujarat/gujarat-assembly-election-2022-dg-vanjara-formed-party-named-praja-vijay-paksha/81898/amp/?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()
soup = BeautifulSoup(response.content, 'html.parser')
headlines = soup.find_all('h1')
for headline in headlines:
    print("Headline:", headline.text.strip())

paragraphs = soup.select('p')
article_text = ""
for paragraph in paragraphs:
  #print(paragraph)
  #print(" -----------------------------------")
  if "देश और दुनिया की ताज़ा खबरें सबसे पहले न्यूज़ 24 पर फॉलो करें न्यूज़ 24 को" in paragraph.get_text():
    break
  if any('\u0900' <= c <= '\u097F' for c in paragraph.get_text()):
    article_text += paragraph.get_text() + "\n"
print(article_text)

Headline: Gujarat Election 2022: ‘प्रजा विजय पक्ष’ नाम से डीजी बंजारा ने बनाई पार्टी, बोले- लोहा ही लोहे को काट सकता है
Gujarat Assembly Election 2022: वंजारा ने कहा कि 27 साल से एक ही पार्टी शासन में है। लंबे समय तक सत्ता में अगर एक ही पार्टी रहती है तो सत्ता भ्रष्ट कर देती है।
वंजारा ने कहा कि 1960 में गुजरात बना। उसके बाद कांग्रेस का शासन रहा। अब 27 साल से एक ही पार्टी शासन में है। लंबे समय तक सत्ता में अगर एक ही पार्टी रहती है तो सत्ता भ्रष्ट कर देती है। पहले यह परिस्थिति कांग्रेस के समय में पैदा हुई थी और अब यह परिस्थिति बीजेपी के लिए लागू हो रही है।
उन्होंने कहा कि जैसे विदेशों में है, उसी तरह से राज्य में दो ही पक्ष है, जो बारी-बारी सत्ता का भोग करें। एक पक्ष 2 साल या 5 साल तक सत्ता में रहे और उसके बाद सत्ता परिवर्तन होना ही चाहिए। अगर ऐसा नहीं होता तो कई बुराइयां पैदा होती है, जो अभी गुजरात में है इसीलिए हम प्रजा विजय पक्ष गुजरात में लेकर आए हैं।
डीजी वंजारा ने कहा कि अब लोगों की मानसिकता देखते हुए सत्ता में मौजूद हिंदुत्ववादी पक्ष के सामने दूसरे हिंदुत्ववादी पक्ष को स्थान देना

# hindi.theprint.in (Working)

In [ ]:
url = 'https://hindi.theprint.in/india/%e0%a4%aa%e0%a4%b6%e0%a5%8d%e0%a4%9a%e0%a4%bf%e0%a4%ae-%e0%a4%b0%e0%a5%87%e0%a4%b2%e0%a4%b5%e0%a5%87-%e0%a4%a8%e0%a5%87-%e0%a4%ae%e0%a5%81%e0%a4%82%e0%a4%ac%e0%a4%88-%e0%a4%a6%e0%a4%bf%e0%a4%b2/326196/?amp=&utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()
soup = BeautifulSoup(response.content, 'html.parser')
headlines = soup.find_all('h1')
for headline in headlines:
    print("Headline:", headline.text.strip())

SSLError: ignored

# hindi.newsbytesapp.com (working)

In [ ]:
url = 'https://hindi.newsbytesapp.com/news/politics/eknath-shinde-share-letter-of-mla-targetting-uddhav-thackrey/story'
response = requests.get(url)

if response.status_code != 200:
    print("Failed to retrieve the webpage.")
    exit()
soup = BeautifulSoup(response.content, 'html.parser')
headlines = soup.find_all('h1')
for headline in headlines:
    print("Headline:", headline.text.strip())

paragraphs = soup.select('p')
article_text = ""
for paragraph in paragraphs:
  #print(paragraph)
  #print(" -----------------------------------")
  if "देश की खबरें पसंद हैं?" in paragraph.get_text() or "राजनीति की खबरें पसंद हैं" in paragraph.get_text() :
    break
  if any('\u0900' <= c <= '\u097F' for c in paragraph.get_text()):
    article_text += paragraph.get_text() + "\n"
#print(article_text)
cleaned_hindi_text = ' '.join(article_text.split())
cleaned_hindi_text = '\n'.join(line.strip() for line in cleaned_hindi_text.splitlines() if line.strip())
cleaned_hindi_text = re.sub(r'\s+([.,;!?])', r'\1', cleaned_hindi_text)

print(cleaned_hindi_text)

Headline: एकनाथ शिंदे ने शेयर किया अपने खेमे के विधायक का पत्र, ठाकरे पर साधा निशाना
उद्धव ठाकरे के खिलाफ बगावत का झंडा उठाने वाले एकनाथ शिंदे ने गुरुवार को एक पत्र शेयर किया है। संजय शिरसाट नामक विधायक ने यह पत्र बालासाहेब ठाकरे को संबोधित करते हुए लिखा है, लेकिन यह उद्धव ठाकरे के नाम है। इसमें लिखा गया है कि पिछले ढाई साल से मुख्यमंत्री उद्धव ठाकरे के आधिकारिक आवास 'वर्षा' के दरवाजे शिवसेना विधायकों के लिए बंद थे। कल सही मायनों में इस बंगले के दरवाजे जनता के लिए खुले हैं। मुख्यमंत्री उद्धव ठाकरे ने कल फेसबुक लाइव पर संबोधन के बाद अपना आधिकारिक आवास खाली कर दिया था और निजी आवास पर लौट आए हैं। संबोधन के दौरान उन्होंने कहा था कि वो इस्तीफा देने को तैयार हैं। शिरसाट ने अपने पत्र में आरोप लगाया है कि 'वर्षा' में प्रवेश पाने के लिए मुख्यमंत्री के आसपास रहने वाले लोगों की मनुहार करनी पड़ती थी, जो कभी चुनाव जीतकर नहीं आए। शिवसेना विधायकों को कभी बंगले में सीधा प्रवेश नहीं मिला। महाराष्ट्र में हाल ही में संपन्न हुए विधान परिषद चुनावों का संदर्भ देते हुए लिखा गया है कि कुछ तथाकथित चाणक्य क्लर्